In [2]:
import Pkg
using Pkg

# Install required packages
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("StatsModels")
Pkg.add("GLM")
Pkg.add("Random")
Pkg.add("MLDataUtils")
Pkg.add("MLBase")
Pkg.add("FixedEffectModels")
Pkg.add("Lasso")
Pkg.add("MLJ")
Pkg.add("DecisionTree")
Pkg.add("RData")
Pkg.add("GLMNet")
Pkg.add("PrettyTables")
Pkg.add("MLJScikitLearnInterface")
Pkg.add("MLJFlux")
Pkg.add("Flux")

    Updating registry at `C:\Users\camif\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\camif\.julia\environments\v1.11\Project.toml`
  [336ed68f] + CSV v0.10.15
  No Changes to `C:\Users\camif\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\camif\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\camif\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\camif\.julia\environments\v1.11\Project.toml`
  [3eaba693] + StatsModels v0.7.7
  No Changes to `C:\Users\camif\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\camif\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\camif\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\camif\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\camif\.julia\environments\v1

In [25]:
using CSV, DataFrames, StatsModels, GLM, Random, RData, MLBase, MLJ, PrettyTables, FixedEffectModels
using MLDataUtils, FixedEffectModels, DecisionTree, Lasso, GLMNet, MLJScikitLearnInterface
using Downloads, Statistics, Flux

# **I. Cleaning and Set-Up**

In [27]:
# URL de tu dataset
url = "https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat"
Downloads.download(url, "penn_jae.dat")
# Leer el archivo
data = CSV.read("penn_jae.dat", DataFrame; delim=' ', ignorerepeated=true)

# Mostrar información básica
println("Número de filas: ", size(data, 1))
println("Número de columnas: ", size(data, 2))
println("Nombres de columnas:")
println(names(data))

first(data, 5)
  

Número de filas: 13913
Número de columnas: 23
Nombres de columnas:
, "agelt35", "agegt54", "durable", "nondurable", "lusd", "husd", "muld"]thrace", "dep", "q1", "q2", "q3", "q4", "q5", "q6", "recall"


Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,10824,0,18,18,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,10635,2,7,3,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
3,10551,5,18,6,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0
4,10824,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
5,10747,0,27,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


In [29]:
# Cleaning and set-up
filter!(row -> row.tg == 0 || row.tg == 4, data)
data.T4 = [row.tg == 4 ? 1 : 0 for row in eachrow(data)]
filter!(row -> row.inuidur1 > 0, data)
data.y = log.(data.inuidur1)
data.dep_0 = [row.dep == 0 ? 1 : 0 for row in eachrow(data)]
data.dep_1 = [row.dep == 1 ? 1 : 0 for row in eachrow(data)]
data.dep_2 = [row.dep == 2 ? 1 : 0 for row in eachrow(data)]

xvars = [
    :female, :black, :othrace,
    :dep_1, :dep_2,
    :q2, :q3, :q4, :q5, :q6,
    :recall, :agelt35, :agegt54,
    :durable, :nondurable, :lusd, :husd
]

# Definir Y, D y X para la estimación
y = data.y
d = data.T4
x = data[:, xvars]

println("\nVariables listas para DML:")
println("Outcome (Y): log(inuidur1)")
println("Treatment (D): T4 (tg==4)")
println("Número de controles (X): ", length(xvars))
println("Dimensiones de X: ", size(x))

# Vista rápida de los datos
first(data, 5)


Variables listas para DML:
Outcome (Y): log(inuidur1)
Treatment (D): T4 (tg==4)
Número de controles (X): 17
Dimensiones de X: (5099, 17)


Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4,y,dep_0,dep_1,dep_2
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64
1,10824,0,18,18,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2.89037,0,0,1
2,10824,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.0,1,0,0
3,10747,0,27,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,3.29584,1,0,0
4,10607,4,9,9,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,2.19722,1,0,0
5,10831,0,27,27,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,3.29584,0,1,0


# **II. Debiased ML**

In [31]:
function training_sample_append(cv_split, test_sample_index)
        training_indices = []
        for vector in cv_split[Not(test_sample_index)]
                training_indices = [training_indices; vector]
        end
        return training_indices, cv_split[test_sample_index]
end

function dml(x, d, y, dreg, yreg, nfold)
        n = length(y)
        cv = [partition(eachindex(y), fill(1/nfold, nfold-1)..., shuffle = true, rng = 1234)...]
        machine_y = machine(yreg, x, y, scitype_check_level=0)
        machine_d = machine(dreg, x, d, scitype_check_level=0)
        y_hat = zeros(n)
        d_hat = zeros(n)

        for fold in 1:nfold
                training_fold, test_fold = training_sample_append(cv, fold)
                y_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_y, rows = training_fold), x[test_fold, :])
                d_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_d, rows = training_fold), x[test_fold, :])
        end

        resy = y .- y_hat
        resd = reshape(d .- d_hat, (n, 1))
        estimate = lm(resd, resy)
        coef_est = GLM.coef(estimate)[1]
        se = GLM.coeftable(estimate).cols[2][1]
        println(" coef (se) = ", coef_est ,"(",se,")")
        return coef_est, se, resy, resd;
end

function summarize(point, stderr, resy, resd, name)
        return DataFrame(
                model = [name],
                estimate = [point], stderr = [stderr], 
                rmse_y = [sqrt(mean(resy .^ 2))], 
                rmse_d = [sqrt(mean(resd .^ 2))]
        )
end

summarize (generic function with 1 method)

In [33]:
LinearRegressor = @load LinearRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = Standardizer() |> LinearRegressor()
yreg = Standardizer() |> LinearRegressor()
result_ols = dml(x, d, y, dreg, yreg, 10)
table_ols = summarize(result_ols..., "OLS")

LassoCVRegressor = @load LassoCVRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
yreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
results_lasso = dml(x, d, y, dreg, yreg, 10)
table_lasso = summarize(results_lasso..., "LassoCV")

RandomForestRegressor = @load RandomForestRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = RandomForestRegressor()
yreg = RandomForestRegressor()
results_rf = dml(x, d, y, dreg, yreg, 10)
table_rf = summarize(results_rf..., "RF")

dreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
results_mix = dml(x, d, y, dreg , yreg, 10)
table_mix = summarize(results_mix..., "RF/LassoCV")

NeuralNetworkRegressor = @load NeuralNetworkRegressor pkg=MLJFlux verbosity=0 
nn_model = NeuralNetworkRegressor(
    builder = MLJFlux.MLP(; hidden=(20,20), σ=relu), 
    epochs = 100,
    batch_size = 32,
    optimiser = Flux.ADAM(0.001),
    rng = 1234
)
dreg = Standardizer() |> nn_model
yreg = Standardizer() |> nn_model
results_nn = dml(x, d, y, dreg, yreg, 10)
table_nn = summarize(results_nn..., "NeuralNet")

[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:linear_regressor, …).
[ In

 coef (se) = -0.06980700224406339(0.03522987047951292)


[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637


 coef (se) = -0.072970735802579(0.03528167358184795)


[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(

...

[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).

 coef (se) = -0.09080696524752567(0.037422782103159594)


[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:neural_network_regressor, …).
[ Info: MLJFlux: converting input data to Float32
Optimising neural net: 100%[=========================] Time: 0:00:22
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:neural_network_regressor, …).
[ Info: MLJFlux: converting input data to Float32
Optimising neural net: 100%[=========================] Time: 0:00:04
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(feature

 coef (se) = -0.07430571641084491(0.034571238367087524)


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,NeuralNet,-0.0743057,0.0345712,1.21673,0.492702


In [35]:
pretty_table([table_ols; table_lasso; table_rf; table_mix; table_nn])

┌────────────┬────────────┬───────────┬─────────┬──────────┐
│      model │   estimate │    stderr │  rmse_y │   rmse_d │
│     String │    Float64 │   Float64 │ Float64 │  Float64 │
├────────────┼────────────┼───────────┼─────────┼──────────┤
│        OLS │  -0.069807 │ 0.0352299 │ 1.19593 │ 0.475255 │
│    LassoCV │ -0.0729707 │ 0.0352817 │ 1.19601 │ 0.474574 │
│         RF │ -0.0953436 │ 0.0348782 │ 1.26721 │ 0.508481 │
│ RF/LassoCV │  -0.090807 │ 0.0374228 │ 1.26879 │ 0.474574 │
│  NeuralNet │ -0.0743057 │ 0.0345712 │ 1.21673 │ 0.492702 │
└────────────┴────────────┴───────────┴─────────┴──────────┘


In [37]:
vcat(table_ols, table_lasso, table_rf, table_mix, table_nn)

Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,OLS,-0.069807,0.0352299,1.19593,0.475255
2,LassoCV,-0.0729707,0.0352817,1.19601,0.474574
3,RF,-0.0953436,0.0348782,1.26721,0.508481
4,RF/LassoCV,-0.090807,0.0374228,1.26879,0.474574
5,NeuralNet,-0.0743057,0.0345712,1.21673,0.492702


## Modelo

Basándome en los resultados, elijo **OLS** y **LassoCV** como los modelos más adecuados.
**OLS** tiene el RMSE más bajo para predecir *Y* (1.196) y un valor muy competitivo para *D* (0.475).
**LassoCV** obtiene resultados prácticamente idénticos, con el RMSE D más bajo de todos los modelos (0.475) y apenas una diferencia mínima en RMSE Y (1.196).
Además, ambos tienen errores estándar muy similares y pequeños (0.035), lo que indica estimaciones precisas y estables.

El **Random Forest** muestra RMSE notablemente más altos tanto para *Y* (1.267) como para *D* (0.508), lo que indica menor capacidad predictiva en este contexto. El **Neural Network** presenta resultados intermedios con RMSE Y de 1.217 y RMSE D de 0.493, superiores a OLS y Lasso pero mejores que RF. Sin embargo, su ganancia no justifica la mayor complejidad del modelo.

Por lo tanto, utilizo **OLS y LassoCV** para estimar el efecto causal.
Los resultados son muy similares:
- **OLS:** coeficiente = -0.0698 (se = 0.0352)
- **LassoCV:** coeficiente = -0.0730 (se = 0.0353)

Esto indica que el tratamiento reduce el logaritmo de la duración del desempleo en aproximadamente 0.07 unidades, un efecto negativo y estadísticamente significativo, consistente entre ambos métodos. La similitud en las estimaciones refuerza la robustez del resultado.

# **III. No cross-fitting**

In [43]:
function dml_no_crossfit(x, d, y, dreg, yreg)
    n = length(y)
    machine_y = machine(yreg, x, y, scitype_check_level=0)
    machine_d = machine(dreg, x, d, scitype_check_level=0)
    
    MLJ.fit!(machine_y)
    MLJ.fit!(machine_d)
    
    y_hat = MLJ.predict(machine_y, x)
    d_hat = MLJ.predict(machine_d, x)
    
    resy = Float64.(y .- y_hat)
    resd = Float64.(reshape(d .- d_hat, (n, 1)))
    estimate = lm(resd, resy)
    coef_est = GLM.coef(estimate)[1]
    se = GLM.coeftable(estimate).cols[2][1]
    
    println("\ncoef (se) = ", round(coef_est, digits=6), " (", round(se, digits=6), ")")
    return coef_est, se, resy, resd
end

println("\nDML sin cross-fitting: OLS")
dreg = Standardizer() |> LinearRegressor()
yreg = Standardizer() |> LinearRegressor()
result_ols_nocv = dml_no_crossfit(x, d, y, dreg, yreg)
table_ols_nocv = summarize(result_ols_nocv..., "OLS")

println("\nDML sin cross-fitting: Lasso")
dreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
yreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
results_lasso_nocv = dml_no_crossfit(x, d, y, dreg, yreg)
table_lasso_nocv = summarize(results_lasso_nocv..., "Lasso")

println("\nDML sin cross-fitting: RF")
dreg = RandomForestRegressor()
yreg = RandomForestRegressor()
results_rf_nocv = dml_no_crossfit(x, d, y, dreg, yreg)
table_rf_nocv = summarize(results_rf_nocv..., "RF")

println("\nDML sin cross-fitting: Neural Network")
nn_model = NeuralNetworkRegressor(
    builder = MLJFlux.MLP(; hidden=(20,20), σ=Flux.relu), 
    epochs = 100,
    batch_size = 32,
    optimiser = Flux.ADAM(0.001),
    rng = 1234
)
dreg = Standardizer() |> nn_model
yreg = Standardizer() |> nn_model
results_nn_nocv = dml_no_crossfit(x, d, y, dreg, yreg)
table_nn_nocv = summarize(results_nn_nocv..., "Neural Network")


DML sin cross-fitting: OLS


[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:linear_regressor, …).



coef (se) = -0.072576 (0.035208)

DML sin cross-fitting: Lasso


[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:lasso_cv_regressor, …).



coef (se) = -0.072853 (0.035168)

DML sin cross-fitting: RF


[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).



coef (se) = -0.0808 (0.035251)

DML sin cross-fitting: Neural Network


[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:neural_network_regressor, …).
[ Info: MLJFlux: converting input data to Float32
Optimising neural net: 100%[=========================] Time: 0:00:02
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: No features to standarize.
└ @ MLJModels C:\Users\camif\.julia\packages\MLJModels\ziReN\src\builtins\Transformers.jl:637
[ Info: Training machine(:neural_network_regressor, …).
[ Info: MLJFlux: converting input data to Float32
Optimising neural net: 100%[=========================] Time: 0:00:01



coef (se) = -0.079191 (0.03527)


Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,Neural Network,-0.0791907,0.0352696,1.14463,0.454306


In [45]:
table_nocv = vcat(table_ols_nocv, table_lasso_nocv, table_rf_nocv, table_nn_nocv)

Row,model,estimate,stderr,rmse_y,rmse_d
,String,Float64,Float64,Float64,Float64
1,OLS,-0.0725756,0.0352081,1.19047,0.473365
2,Lasso,-0.0728532,0.0351679,1.19052,0.473923
3,RF,-0.0807997,0.0352512,1.07728,0.42779
4,Neural Network,-0.0791907,0.0352696,1.14463,0.454306


In [49]:
pretty_table(table_nocv, formatters = ft_printf("%0.4f"))

┌────────────────┬──────────┬─────────┬─────────┬─────────┐
│          model │ estimate │  stderr │  rmse_y │  rmse_d │
│         String │  Float64 │ Float64 │ Float64 │ Float64 │
├────────────────┼──────────┼─────────┼─────────┼─────────┤
│            OLS │  -0.0726 │  0.0352 │  1.1905 │  0.4734 │
│          Lasso │  -0.0729 │  0.0352 │  1.1905 │  0.4739 │
│             RF │  -0.0808 │  0.0353 │  1.0773 │  0.4278 │
│ Neural Network │  -0.0792 │  0.0353 │  1.1446 │  0.4543 │
└────────────────┴──────────┴─────────┴─────────┴─────────┘


## Modelo (Sin Cross-Fitting)

Basándome en los resultados sin cross-fitting, elijo **Random Forest** como el modelo más adecuado.
**Random Forest** presenta el RMSE más bajo tanto para predecir *Y* (1.077) como para *D* (0.428), superando claramente a los demás métodos.
Su error estándar (0.0353) es prácticamente idéntico al de **OLS** y **Lasso**, lo que indica una estimación precisa y estable.

**OLS** y **Lasso** también muestran buenos resultados, con valores de *RMSE Y* de 1.191 y *RMSE D* alrededor de 0.47, pero su desempeño es notablemente inferior al de **Random Forest**.
El **Neural Network** tiene un *RMSE Y* intermedio (1.145) y un error estándar similar (0.0353), aunque tanto su *RMSE Y* como su *RMSE D* (0.454) son superiores a los de Random Forest.

Por lo tanto, utilizo **Random Forest** para estimar el efecto causal sin cross-fitting.
El resultado obtenido es:
- **Random Forest:** coeficiente = -0.0808 (se = 0.0353)

Esto indica que el tratamiento reduce el logaritmo de la duración del desempleo en aproximadamente 0.08 unidades, un efecto negativo y estadísticamente significativo.

## Comparación: Con vs. Sin Cross-Fitting

### 1. ¿Qué puedo decir sobre el RMSE para predecir *y* y *d*?

Los RMSE sin cross-fitting son más bajos, especialmente en los modelos más flexibles.
Por ejemplo:
- **Random Forest:** el RMSE de *Y* baja de **1.267** a **1.077** y el de *D* de **0.508** a **0.428**.
- **Neural Network:** el RMSE de *Y* disminuye de **1.217** a **1.145**, y el de *D* de **0.493** a **0.454**.
- En **OLS** y **Lasso**, los cambios son mínimos: RMSE Y se mantiene prácticamente igual (1.196 vs 1.191) y RMSE D también es muy similar (0.475 vs 0.474).

En general, los modelos más complejos (RF y NN) muestran las reducciones más grandes, mientras que los lineales se mantienen estables.

### 2. ¿Por qué una función da RMSE menor que la otra?

Los RMSE son menores sin cross-fitting porque hay **sobreajuste (overfitting)**.
En ese caso, el modelo se entrena y se evalúa sobre los mismos datos, por lo que termina "aprendiéndose" el ruido del conjunto de entrenamiento.
Esto hace que los errores parezcan más pequeños, pero en realidad el modelo no está generalizando bien.

Los métodos más flexibles, como **Random Forest** y **Neural Network**, tienen más capacidad para ajustarse a los datos, por eso muestran las mayores caídas en RMSE.
Con **cross-fitting**, los modelos predicen sobre datos que no vieron durante el entrenamiento, así que los RMSE son más altos, pero **más realistas y confiables**.

### 3. ¿Qué problema hay si estimamos sin cross-fitting?

El problema principal es que **las estimaciones del efecto causal se sesgan**.
Sin cross-fitting, los residuales están correlacionados con las predicciones porque se usan los mismos datos para entrenar y estimar.
Esto hace que el coeficiente del tratamiento no sea totalmente confiable y que los errores estándar estén mal calculados.

En cambio, **el cross-fitting evita ese sesgo** al usar muestras distintas para entrenar y para predecir, garantizando que los residuales sean independientes.
Así, los resultados son más válidos para inferencia causal y podemos usar modelos flexibles sin perder rigor estadístico.RetryClaude can make mistakes. Please double-check responses.